In [ ]:
# Predicción de variables numéricas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pandas.tseries.offsets import QuarterEnd

# Cargar archivo Excel
file_path = '/mnt/data/Internet.xlsx'
excel_data = pd.ExcelFile(file_path)

def predict_next_quarters(excel_data, forecast_periods=4):
    results = {}
    for sheet_name in excel_data.sheet_names:
        print(f"\n--- Predicción para la hoja: {sheet_name} ---")
        df = excel_data.parse(sheet_name).dropna()

        # Selección de variables numéricas
        numeric_cols = df.select_dtypes(include=[np.number]).columns

        for col in numeric_cols:
            print(f"\nVariable: {col}")
            ts = df[col]

            # Comprobación de índice temporal
            if not pd.api.types.is_datetime64_any_dtype(df.index):
                if 'date' in df.columns or 'fecha' in df.columns:
                    df['date'] = pd.to_datetime(df['date'], errors='coerce')
                    df = df.set_index('date').sort_index()

            # Asegurar que el índice sea trimestral
            ts = ts.asfreq('Q', method='pad')

            # Descomposición de la serie temporal
            decomposition = seasonal_decompose(ts, model='additive')
            decomposition.plot()
            plt.show()

            # Modelo de Holt-Winters
            model = ExponentialSmoothing(ts, seasonal='additive', seasonal_periods=4)
            fit = model.fit()

            # Predicciones para los próximos trimestres
            forecast = fit.forecast(forecast_periods)
            print(f"Predicciones para los próximos {forecast_periods} trimestres:\n", forecast)

            # Gráfica de predicción
            plt.figure(figsize=(10, 6))
            plt.plot(ts, label='Datos históricos')
            plt.plot(forecast, label='Predicción', color='orange')
            plt.title(f"Predicción para {col} en {sheet_name}")
            plt.legend()
            plt.show()

predict_next_quarters(excel_data)